### Sequence of wayoints in joint space and open-close movement of the gripper after each waypoint

**requirements:**

- beware that RobotControl package is required to open and close the gripper
- You must launch: ros2 launch xarm_api lite6_driver.launch.py robot_ip:=192.169.1.190

In [ ]:
import os
import math
import sys
import yaml
import time
import rclpy
import numpy as np
from geometry_msgs.msg import PoseStamped, Pose
#for moveit2
from moveit.planning import MoveItPy
from moveit.core.robot_state import RobotState
from moveit.core.kinematic_constraints import construct_joint_constraint
from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory
# For gripper control
from xarm_msgs.srv import MoveCartesian,MoveJoint,MoveHome,Call,SetInt16,SetInt16ById 
from lite6_arm_interface.srv import FindBoxes
from lite6_arm_interface.msg import Box
from xarm_msgs.msg import RobotMsg

from lite6_arm.robot_control import RobotControl
import random 


**we need to specify our moveit_py config at the top of each notebook we use.**
**this is since we will start spinning a moveit_py node within this notebook.**

In [ ]:
moveit_config = (
        MoveItConfigsBuilder(robot_name="UF_ROBOT", package_name="lite6_enrico")
        .robot_description_semantic(file_path="config/UF_ROBOT.srdf")
        .trajectory_execution(file_path="config/moveit_controllers.yaml")
        .robot_description(file_path="config/UF_ROBOT.urdf.xacro")
        .moveit_cpp(
            file_path=get_package_share_directory("lite6_moveit_demos")
            + "/config/moveit_cpp.yaml"
        )
        .to_moveit_configs()
    ).to_dict()

In [ ]:
rclpy.init()

In [ ]:
# Intatiate RobotControl for gripper control
node = RobotControl()
node.open_gripper()

In [ ]:
#istantiate MoveitPy object 
moveit = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
lite6 = moveit.get_planning_component("lite6_arm")

In [ ]:
lite6.set_start_state_to_current_state()

# set constraints message
joint_values = {
    "joint1": math.radians(0),
    "joint2": math.radians(10.4),
    "joint3": math.radians(31.1),
    "joint4": math.radians(-1.5),
    "joint5": math.radians(21.5),
    "joint6": math.radians(1.3),
}
robot_state = RobotState(moveit.get_robot_model())
robot_state.joint_positions = joint_values
joint_constraint = construct_joint_constraint(
    robot_state=robot_state,
    joint_model_group=moveit.get_robot_model().get_joint_model_group("lite6_arm"),
)
lite6.set_goal_state(motion_plan_constraints=[joint_constraint])

# plan to goal
plan_result = lite6.plan()

# execute the plan
if plan_result:
    robot_trajectory = plan_result.trajectory
    moveit.execute(robot_trajectory, controllers=[])

In [ ]:
node.open_gripper()
time.sleep(1.0)

In [ ]:
node.close_gripper()

In [ ]:
lite6.set_start_state_to_current_state()
lite6.set_goal_state(configuration_name="Ready")
# plan to goal
plan_result = lite6.plan()

# execute the plan
if plan_result:
    robot_trajectory = plan_result.trajectory
    moveit.execute(robot_trajectory, controllers=[])

**Create a node that give to the joint randomic values**

the only problem is: I can't put just random values for the joint, they have to be random VALID. 
How can I actually do that?